In [ ]:
import numpy as np
from pydantic import BaseModel, ConfigDict
from typing import Sequence
from typing import Annotated, Literal, TypeVar
import numpy as np
import numpy.typing as npt


DType = TypeVar("DType", bound=np.generic)
Vector3D = Annotated[npt.NDArray[DType], Literal[3]]


class Stone(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    id: int
    pos: Vector3D[np.int64]
    speed: Vector3D[np.int64]

    def at(self, t: float) -> Vector3D[np.float64]:
        return self.pos + self.speed * t

In [ ]:
stones = []
with open("day24_input.txt") as file:
    for i, line in enumerate(file):
        pos, speed = line.strip().split(" @ ")
        pos = np.array([int(num) for num in pos.split(",")])
        speed = np.array([int(num) for num in speed.split(",")])
        stones.append(Stone(id=i, pos=pos, speed=speed))

# Part 1


In [ ]:
def intersects_2d(stone1: Stone, stone2: Stone) -> tuple[float, float] | None:
    """a + b*t = c + d*s can be rewritten as A * [t, s] = b"""
    A = np.stack([stone1.speed[:2], -stone2.speed[:2]], axis=1)
    b = stone2.pos[:2] - stone1.pos[:2]
    try:
        t, s = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        print(f"Stone {stone1.id} and {stone2.id} do not intersect.")
        return None

    if s < 0 or t < 0:
        return None

    intersect = stone1.pos + stone1.speed * t
    return intersect[:2]

In [ ]:
import itertools

low = 200000000000000
high = 400000000000000

num_inside = 0
for stone1, stone2 in itertools.combinations(stones, 2):
    intersect = intersects_2d(stone1, stone2)
    if intersect is None:
        continue
    if ((low <= intersect) & (intersect <= high)).all():
        num_inside += 1

print("Answer:", num_inside)

# Part 2


The rock needs to hit three stones at times 1, 2, and 3:

$$
x_0 + v_0 t_1 = x_1 + v_1 t_1 \\
x_0 + v_0 t_2 = x_2 + v_2 t_2 \\
x_0 + v_0 t_3 = x_3 + v_3 t_3
$$

This is a system of nine equations and nine unknowns. They can be rewritten as:

$$
x_0 = x_1 + (v_1 - v_0) t_1 \\
x_0 = x_2 + (v_2 - v_0) t_2 \\
x_0 = x_3 + (v_3 - v_0) t_3 \\
$$

which means that if we subtract $v_0$ from each stone-speed, all stones will pass
through the same coordinate at some time. By guessing values for $v_0$, we can reuse the
code from part 1 to find the times when the stones hit $x_0$.

Another approach could be to rewrite the equations as:

$$
x_0 - x_1 = (v_1 - v_0) t_1 \\
x_0 - x_2 = (v_2 - v_0) t_2 \\
x_0 - x_3 = (v_3 - v_0) t_3 \\
$$

which means that for any stone $i$, $x_0 - x_i$ is parallel to $v_i - v_0$. We can then
use properties of parallel lines (e.g. cross-product is zero) to find $x_0$ and $v_0$.
Since there is now only 6 unknowns, it shold be sufficient to use 2 stones to
algebraically find the solution.


In [ ]:
def intersects_3d(stone1: Stone, stone2: Stone) -> np.ndarray | None:
    """a + b*t = c + d*s can be rewritten as A * [t, s] = b"""
    A = np.stack([stone1.speed, -stone2.speed], axis=1)
    b = stone2.pos - stone1.pos
    # The system is over-determined, with 3 equations and 2 unknowns.
    (t, s), _, rank, _ = np.linalg.lstsq(A, b, rcond=None)

    if s < 0 or t < 0:
        return None

    intersect = stone1.pos + stone1.speed * t
    return intersect

Idea from [here](https://www.reddit.com/r/adventofcode/comments/18pnycy/comment/keqf8uq/?utm_source=share&utm_medium=web2x&context=3) to reduce the search space for possible $v_0$ values.

Consider the x-direction first: If two hailstones have the same x-speed `v`, the
x-distance `dist` between them will always remain the same. Assume the rock has just hit
the first stone at `t1`. It now needs to move `x2 - x1` to hit the second rock at `t2`.
But the second rock is moving at a speed of `v`.

`(rock_speed - v) * (t2 - t1) = x2 - x1`

Assuming the rock hits the stones at integer times, `t1 - t2` is always an integer, the
following equation must be true:

`(x2 - x1) % (rock_speed - v) = 0`


In [ ]:
from collections import Counter


def find_valid_speeds(dim: int) -> set[int]:
    rock_speed_range = range(-1000, 1000)

    # Find a collection of stones with the same speed in the given dimension
    counter = Counter([stone.speed[dim] for stone in stones])
    common_speed = counter.most_common(1)[0][0]
    useful_stones = [stone for stone in stones if stone.speed[dim] == common_speed]

    # For all stones with this speed, find the possible rock speeds that are valid for
    # all of the stones.
    all_possible_speeds = []
    for stone1, stone2 in zip(useful_stones, useful_stones[1:]):
        possible_speeds = set()
        posdiff = stone2.pos[dim] - stone1.pos[dim]
        stone_speed = stone1.speed[dim]
        for rock_speed in rock_speed_range:
            if rock_speed == stone_speed:
                continue
            if (posdiff % (rock_speed - stone_speed)) == 0:
                possible_speeds.add(rock_speed)
        all_possible_speeds.append(possible_speeds)

    return set.intersection(*all_possible_speeds)

In [ ]:
import random

for rock_speed in itertools.product(*(find_valid_speeds(dim) for dim in range(3))):
    # Pick 3 random stones
    samples = [sample.model_copy(deep=True) for sample in random.sample(stones, 3)]

    # Adjust the speed of the stones, so they are relative to the rock
    for sample in samples:
        sample.speed -= rock_speed

    # See if the 3 stones intersect at the same point
    intersect1 = intersects_3d(samples[0], samples[1])
    intersect2 = intersects_3d(samples[0], samples[2])
    if intersect1 is None or intersect2 is None:
        continue

    # If they do, we have a solution
    if np.allclose(intersect1, intersect2):
        intersect = intersect1.round().astype(int)
        break
else:
    raise RuntimeError("No solution found")

print(intersect, rock_speed)
print("Answer:", sum(intersect))